# minGPT

**Note:** The `autoreload` extension allows the interpreter to reload modules every time a cell is executed. This is useful when editing the code in a module. The following cell enables the extension and downloads the minGPT package from Github. You can now double-click on a file like model.py, edit its contents, and press Ctrl+S to save it. If you then re-run the notebook cells, including those that create an object of the corresponding class, you will see the changes reflected. Note that the next cell should *only be executed once*, as running `pip install` again will overwrite the modified contents of the module.

Recall that changes in the files (except the notebook itself) are not persistent unless you connect them to your Google Drive account.

In [ ]:
%load_ext autoreload
%autoreload 2
%pip install -e 'git+https://github.com/karpathy/minGPT.git@37baab71b9abea1b76ab957409a1cc2fbfba8a26#egg=mingpt'

# Fix this issue: https://github.com/karpathy/minGPT/issues/120
!sed -i '200s/.*/        assert len(keys) == len([k for k in sd if not k.endswith(".attn.bias")])/' /content/src/mingpt/mingpt/model.py


Obtaining mingpt from git+https://github.com/karpathy/minGPT.git@37baab71b9abea1b76ab957409a1cc2fbfba8a26#egg=mingpt
  Cloning https://github.com/karpathy/minGPT.git (to revision 37baab71b9abea1b76ab957409a1cc2fbfba8a26) to ./src/mingpt
  Running command git clone --filter=blob:none --quiet https://github.com/karpathy/minGPT.git /content/src/mingpt
  Running command git rev-parse -q --verify 'sha^37baab71b9abea1b76ab957409a1cc2fbfba8a26'
  Running command git fetch -q https://github.com/karpathy/minGPT.git 37baab71b9abea1b76ab957409a1cc2fbfba8a26
  Resolved https://github.com/karpathy/minGPT.git to commit 37baab71b9abea1b76ab957409a1cc2fbfba8a26
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mingpt


Add module's location to PYTHONPATH, which tells your Python interpreter where to search modules for. The previous `pip install -e` changes the variable in a subshell and the interpreter is therefore not aware of the updated value.

In [ ]:
import sys
sys.path.append('/content/src/mingpt')

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

In [ ]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'
device = 'cuda'

In [ ]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

number of parameters: 124.44M


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:

def generate(prompt='', num_samples=10, steps=20, do_sample=True):

    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '':
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']

    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)

    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)


In [ ]:
generate(prompt='Andrej Karpathy, the Earth representative on', num_samples=10, steps=20)

downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json to /root/.cache/mingpt/encoder.json
downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe to /root/.cache/mingpt/vocab.bpe
--------------------------------------------------------------------------------
Andrej Karpathy, the Earth representative on NASA's Curiosity, told the BBC: "I have to say, it really is a massive job
--------------------------------------------------------------------------------
Andrej Karpathy, the Earth representative on the ground in central and southeastern Central Asia, told a news conference in China after the ceremony.

--------------------------------------------------------------------------------
Andrej Karpathy, the Earth representative on the ground, was taken to a hospital.

She was found unconscious and in a black room
--------------------------------------------------------------------------------
Andrej Karpathy, the Earth representat